In [1]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
WEBSITE = "http://books.toscrape.com/catalogue/page-1.html"

In [8]:
html_content = requests.get(WEBSITE).text
soup = BeautifulSoup(html_content)

In [4]:
def get_title(soup):    
    title=[]
    for i in soup.findAll('h3'):
        for j in i.findAll('a'):
            title.append(j.get('title'))
    return title

In [5]:
def get_price(soup):
    prices=[]
    for price in soup.findAll('p',attrs={'class':'price_color'}):
        prices.append(price.text.split('£')[1])
    return prices
    

In [6]:
def images(soup):
    image_links=[]
    for img_link in soup.findAll('img',attrs={'class':'thumbnail'}):
        image_links.append('http://books.toscrape.com/'+img_link.get('src'))
    return image_links

In [77]:
titles=[]
prices=[]
images_links=[]
details = {'UPC':[],'Product Type':[],'Price (excl. tax)':[],'Price (incl. tax)':[],'Tax':[],'Availability':[],'Number of reviews':[]}
for pg_no in range(1,51):    
    link = "http://books.toscrape.com/catalogue/page-"+str(pg_no)+".html"
    html_content = requests.get(link).text
    soup = BeautifulSoup(html_content)
    
    #get_title
    titles.extend(get_title(soup))
    
    #get_price
    prices.extend(get_price(soup))
    
    #images
    images_links.extend(images(soup))
    
    for i in soup.findAll('h3'):
        for j in i.findAll('a'):
            book_content = requests.get('http://books.toscrape.com/catalogue/'+j.get('href')).text
            book_soup = BeautifulSoup(book_content)
            for tr in book_soup.findAll('tr'):
                for th in tr.findAll('th'):
                    for td in tr.findAll('td'):
                        details[th.text].append(td.text)
    
    
    

In [78]:
scraped_details = pd.DataFrame({'BOOK_TITLE':titles,'PRICE (£)':prices, 'IMAGES':images_links})
book_info = pd.DataFrame(details)

In [86]:
final_scrape = pd.concat([scraped_details,book_info],axis=1)

In [87]:
final_scrape.head()

,BOOK_TITLE,PRICE (£),IMAGES,UPC,Product Type,Price (excl. tax),Price (incl. tax),Tax,Availability,Number of reviews
0,A Light in the Attic,51.77,http://books.toscrape.com/../media/cache/2c/da...,a897fe39b1053632,Books,Â£51.77,Â£51.77,Â£0.00,In stock (22 available),0
1,Tipping the Velvet,53.74,http://books.toscrape.com/../media/cache/26/0c...,90fa61229261140a,Books,Â£53.74,Â£53.74,Â£0.00,In stock (20 available),0
2,Soumission,50.10,http://books.toscrape.com/../media/cache/3e/ef...,6957f44c3847a760,Books,Â£50.10,Â£50.10,Â£0.00,In stock (20 available),0
3,Sharp Objects,47.82,http://books.toscrape.com/../media/cache/32/51...,e00eb4fd7b871a48,Books,Â£47.82,Â£47.82,Â£0.00,In stock (20 available),0
4,Sapiens: A Brief History of Humankind,54.23,http://books.toscrape.com/../media/cache/be/a5...,4165285e1663650f,Books,Â£54.23,Â£54.23,Â£0.00,In stock (20 available),0


In [88]:
final_scrape.to_csv(os.getcwd()+'//scraped.csv')